In [1]:
import os

os.environ["LD_LIBRARY_PATH"] = "/home/renan/MATLAB/R2021b/runtime/glnxa64/";

from myPackages import myDehankelization
from myPackages import myDehankelization, myHankelization
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matlab
import pandas as pd
import ecg_plot
from os.path import exists
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'
import csv

import warnings
warnings.filterwarnings('ignore')

/home/renan/Área de Trabalho/research-project/py37/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
experiment = '9. 07-06'
rank = '5'

In [3]:
%matplotlib agg
aldebaran_df = pd.read_pickle(f'../workdata/aldebaran/aldebaran.pkl')
recordings = os.listdir(f'../workdata/saga/{experiment}/rank_{rank}')

if not os.path.exists(f'../workdata/aioria/{experiment}/'):
    os.makedirs(f'../workdata/aioria/{experiment}/') 
    
for rec_path in recordings:
    row = {}
    rec_id = int(rec_path.split('.')[0])
    row['id'] = rec_id
    
    # Create 
    if not os.path.exists(f'../workdata/aioria/{experiment}/{rec_id}/'):
        os.makedirs(f'../workdata/aioria/{experiment}/{rec_id}/') 
    
    # Save ecg 12 lead image
    ecg = aldebaran_df.loc[[rec_id]]['data'].values[0]
    ecg_plot.plot(ecg/1000, sample_rate=100)
    ecg_plot.save_as_png('ecg', f'../workdata/aioria/{experiment}/{rec_id}/')
    
    row['diag'] = aldebaran_df.loc[[rec_id]]['diagnostic'].values[0]

    # Open output from decomposition BTD
    with open(f'../workdata/saga/{experiment}/rank_5/{rec_path}', 'rb') as f:
        output_btd = np.load(f, allow_pickle=True)

    n_sources = len(output_btd)
    
    total_fdom = 0
    for i in range(0, n_sources):
        fig, axs = plt.subplots(ncols=2)
        a = np.array(output_btd[i][0])
        b = np.array(output_btd[i][1]).T
        mixing_matrix = np.dot(a, b)
        my_dehankelization = myDehankelization.initialize()
        m_hankel = matlab.double(mixing_matrix.tolist(), size=mixing_matrix.shape)
        source = my_dehankelization.dehankelization(m_hankel)
        my_dehankelization.terminate()
        x = np.linspace(0, 1, 100, endpoint=False)
        source = np.array(source).T[0]
        
        sns.lineplot(x, source/1000, ax=axs[0])
        axs[0].set(ylim=(-0.1, 0.1))
        
        fourier = np.fft.fft(source)
        frequencies = np.fft.fftfreq(fourier.size, d=x[1]-x[0])
        
        p_fourier = abs(fourier[np.where(frequencies >= 0)])
        p_frequencies = frequencies[np.where(frequencies >= 0)] 
        
        f_dominant = p_frequencies[np.argmax(np.abs(p_fourier))]
        
        if f_dominant >= 3 and f_dominant <=9:
            total_fdom = total_fdom + 1
            
        sns.lineplot(p_frequencies, np.abs(p_fourier), ax=axs[1]).set(title=f'f_dom: {f_dominant}')
        axs[1].set(ylim=(0, 1000))
        fig.savefig(f'../workdata/aioria/{experiment}/{rec_id}/source_{i}.png')
        
        row[f'fdom_src{i}'] = f_dominant
        
    row['total'] = total_fdom
    
    file_exists = exists(f'../workdata/aioria/{experiment}/results.csv')
    
    with open(f'../workdata/aioria/{experiment}/results.csv', 'a') as f:
        headers = row.keys()
        writer = csv.DictWriter(f, delimiter=',', lineterminator='\n', fieldnames=headers)
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)